In [ ]:
!pip install larq

In [ ]:
import tensorflow as tf
import larq as lq
print(tf.__version__)
import numpy as np
import pickle as pickle_rick
from matplotlib import pyplot as plt

print(tf.test.is_built_with_cuda())
from tensorflow.python.client import device_lib
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
print(get_available_devices()) 
import larq as lq

from art.estimators.classification import TensorFlowV2Classifier
from art.attacks.evasion import FastGradientMethod, CarliniLInfMethod


In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# Normalize pixel values to be between -1 and 1
train_images, test_images = train_images / 127.5 - 1, test_images / 127.5 - 1

In [ ]:
# All quantized layers except the first will use the same options
kwargs = dict(input_quantizer="ste_sign",
              kernel_quantizer="ste_sign",
              kernel_constraint="weight_clip")

model = tf.keras.models.Sequential()

# In the first layer we only quantize the weights and not the input
model.add(lq.layers.QuantConv2D(32, (3, 3),
                                kernel_quantizer="ste_sign",
                                kernel_constraint="weight_clip",
                                use_bias=False,
                                input_shape=(28, 28, 1)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.BatchNormalization(scale=False))

model.add(lq.layers.QuantConv2D(64, (3, 3), use_bias=False, **kwargs))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.BatchNormalization(scale=False))

model.add(lq.layers.QuantConv2D(64, (3, 3), use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(tf.keras.layers.Flatten())

model.add(lq.layers.QuantDense(64, use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(lq.layers.QuantDense(10, use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(tf.keras.layers.Activation("softmax"))
loss ='sparse_categorical_crossentropy'

In [ ]:
lq.models.summary(model)


In [ ]:
model.compile(optimizer='adam',
              loss = loss,
              metrics=['accuracy'])

model.fit(train_images, train_labels, batch_size=64, epochs=10)

test_loss, test_acc = model.evaluate(test_images, test_labels)



In [ ]:
print(f"Test accuracy {test_acc * 100:.2f} %")

In [ ]:
pickle_rick.dump(model, open("LqModel", "wb"))